# Analyse relation of input parameters to predicted dmg by RF


In [8]:
import os
import rioxarray as rioxr
import xarray as xr
# import cftime

import re
import dask
import pandas as pd 
import geopandas as gpd
# import matplotlib.pyplot as plt
import numpy as np
import glob
# import matplotlib.patches as mpatches
# from sklearn.metrics import r2_score
# import numpy.polynomial.polynomial as poly

# import rasterio as rio

# import pandas as pd 

# import seaborn as sns

# Import user functions
# get_tilelist_region, load_tile_yxt, clip_da_to_iceshelf, 
# load_tiles_region_multiyear, aggregate_region_ds_iceshelf , 
# remove_nanpx_multivar, fill_nan_cdata, reproject_match_grid
# import myFunctions as myf 
import nbFunctions as myf

homedir = '/Users/tud500158/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - TUD500158/'


## Load observational data to find min-max ranges

In [9]:

def load_nc_sector_years( path2data, sector_ID, year_list=None, varName=None ):
    ''' Load all/selected annual netCDF files of a variable for one sector'''

    ## get filelist of variable for current sector
    filelist_dir =  glob.glob( os.path.join(path2data, f'*_sector-{sector_ID}_*.nc') )
    filelist_var_all = [file for file in filelist_dir if varName in file]
    filelist_var_all.sort()

    ## select files for all/specified years 
    if year_list is None: # all years
        ## load list of files
        filenames = [os.path.basename(file) for file in filelist_var_all]
        ## retrieve available years from filenames
        year_list = [int( re.search(r'\d{4}', file).group()) for file in filenames]
        filelist_var = filelist_var_all.copy()

    else: # filter filelist for desired year
        filelist_var=[]
        for year in year_list:
            filelist_yr = [file for file in filelist_var_all if str(year) in os.path.basename(file)]
            # print(filelist_yr)
            if not filelist_yr:
                raise ValueError(f'Could not find year {year}')
            filelist_var.append(filelist_yr)

    ## Open dataset(s)

    try: # read all years at once
        region_ds = (xr.open_mfdataset(filelist_var ,
                    combine='nested', concat_dim='time',
                    compat='no_conflicts',
                    preprocess=myf.drop_spatial_ref)
            .rio.write_crs(3031,inplace=True)
            .assign_coords(time=year_list) # update year values (y,x,time)
        )
    except ValueError: # read year by year, then concatenate
        region_list = []
        for file in filelist_var:
            yr = int( re.search(r'\d{4}', os.path.basename(file[0])).group()) 
            # print(yr)
            with xr.open_mfdataset(file) as ds:
                try:
                    ds.assign_coords(time=yr)
                except: pass
                region_list.append(ds.rio.write_crs(3031,inplace=True))
        region_ds = xr.concat(region_list,dim='time')  
        # print(region_ds.coords) 
    return region_ds
        

In [10]:

''' ----------------------
Load data: shapefiles 
------------------------- '''
# sectors of interest for AIS

sector_path = os.path.join(homedir, 'QGis/data_NeRD/AIS_outline_sectors.shp')
sector_poly = gpd.read_file(sector_path)
sector_ID_list = sector_poly['sector_ID'].to_list()
sector_ID_list.sort()
sector_ID_list

sector_ID_list = ['ASE', 'BSE', 'EIS', 'RS', 'WIS-a', 'WIS-b', 'WS']

# measures ice shelves
iceshelf_path_meas = os.path.join(homedir, 'QGis/Quantarctica/Quantarctica3/Glaciology/MEaSUREs Antarctic Boundaries/IceShelf/IceShelf_Antarctica_v02.shp')
iceshelf_poly_meas = gpd.read_file(iceshelf_path_meas)
iceshelf_polygon_gpd = iceshelf_poly_meas.drop(['testField','TYPE'],axis=1)

In [ ]:

## data settings
path2data = os.path.join(homedir, 'Data/NERD/data_predictor/data_sector/velocity_rema/')
ksize = 20
length_scales = ['1px']
year_list=['2015','2016','2017','2018']

In [17]:


''' ----------------------
Load data: netCDFs per region, per variable
------------------------- '''

region_ds_list = []
df_min_list=[]
df_max_list=[]
for sector_ID in sector_ID_list: 

    print('----\n Loading netCDF for region ', sector_ID)


    ''' -------------------
    Load all variables from individual netCDF files 
    Expecting data directory to contain netCDFs per sector per training variable per year. 
    ----------------------- '''
    region_ds_varlist=[]
    for var in ['vx','vy']: # base variables to read, from which all other training features are calculated
        region_var = load_nc_sector_years( path2data, sector_ID, varName=var, year_list=year_list) # load all available years
        region_ds_varlist.append(region_var)
    # load rema (only 1 year)
    region_var = load_nc_sector_years( path2data, sector_ID, varName='rema', year_list=['0000']) # , year_list=years_train)
    region_ds_varlist.append(region_var)
    # combine to single dataset
    region_ds = xr.combine_by_coords(region_ds_varlist)
    print('Loaded variables: \n', list(region_ds.keys()) )#, region_ds.coords)
    

    ''' --------------------------------------
    Repeat temporally static variable (REMA) to even out dataset dimension
    This drops time=0
    ------------------------------------------ '''

    region_ds = myf.repeat_static_variable_timeseries( region_ds , 'rema' )


    ''' ----------------
    Downsample observation data ( 400m to 8000m )
    --------------------'''

    if ksize:
        dx = int(region_ds.rio.resolution()[0])
        dy = int(region_ds.rio.resolution()[1])
        if np.abs(dx) != np.abs(dy):
            print("Warning: x and y resolution are not the same; {} and {} -- code update required".format(np.abs(dx), np.abs(dy) ))

        # with dask.config.set(**{'array.slicing.split_large_chunks': True}): # gives error?
        with dask.config.set(**{'array.slicing.split_large_chunks': False}): ## accept large chunks; ignore warning
            region_ds = myf.downsample_dataArray_withoutD0(region_ds, ksize=ksize, 
                                        boundary_method='pad',downsample_func='mean', skipna=False)
        new_res = ksize*400
        print('.. resolution {}m downsampled to {}m'.format(dx, new_res))


        # Check if grid resolution is regular (otherwise, adjust)
        if np.abs(int(region_ds.rio.resolution()[0]) ) != np.abs( int(region_ds.rio.resolution()[1]) ):
            print( "x and y resolution are not the same; {} and {} -- resample to regular grid of {}m".format(
                        np.abs(int(region_ds.rio.resolution()[0])), 
                        np.abs(int(region_ds.rio.resolution()[1])), new_res ))
            
            grid_dummy = myf.make_regular_grid_for_ds(region_ds, grid_res=new_res)
            region_ds.rio.write_crs(3031, inplace=True)
            region_ds = myf.reproject_match_grid( grid_dummy, region_ds ) 


    ''' ------------
    Calculate velocity and strain for (downsampled) data
    ---------------- '''
    # calculate velocity, strain components and temporal velo/strain change
    data_velo_strain, region_ds_roll = myf.calculate_velo_strain_features(region_ds, 
                                                velocity_names=('vx','vy'), 
                                                length_scales=length_scales)
    region_ds = xr.merge([region_ds, data_velo_strain])
    region_ds = xr.merge([region_ds, region_ds_roll])


    ''' --------------------------------------
    Clip to ice shelf
    ------------------------------------------ '''

    ''' Clip to iceshelf '''
    iceshelf_polygon_gpd = iceshelf_poly_meas.drop(['testField','TYPE'],axis=1)
    region_ds  = region_ds.rio.clip( iceshelf_polygon_gpd.geometry, iceshelf_polygon_gpd.crs, drop=False, invert=False)

    region_ds_list.append(region_ds)



----
 Loading netCDF for region  ASE
Loaded variables: 
 ['rema', 'vx', 'vy']
..Downsampling data 20x20 pxs
.. resolution 400m downsampled to 8000m
x and y resolution are not the same; 7979 and 7991 -- resample to regular grid of 8000m
.. calculated strain variables  ['emax_1px', 'emin_1px', 'e_eff_1px', 'elon_1px', 'etrans_1px', 'eshear_1px']
----
 Loading netCDF for region  BSE
Loaded variables: 
 ['rema', 'vx', 'vy']
..Downsampling data 20x20 pxs
.. resolution 400m downsampled to 8000m
x and y resolution are not the same; 7979 and 7984 -- resample to regular grid of 8000m
.. calculated strain variables  ['emax_1px', 'emin_1px', 'e_eff_1px', 'elon_1px', 'etrans_1px', 'eshear_1px']
----
 Loading netCDF for region  EIS
Loaded variables: 
 ['rema', 'vx', 'vy']
..Downsampling data 20x20 pxs
.. resolution 400m downsampled to 8000m
x and y resolution are not the same; 7999 and 7990 -- resample to regular grid of 8000m
.. calculated strain variables  ['emax_1px', 'emin_1px', 'e_eff_1px', 'e

In [13]:

df_min_list=[]
df_max_list=[]
region_q5_list=[];region_q95_list=[];region_med_list=[]
for sector_ID,region_ds in zip(sector_ID_list,region_ds_list): 
    print('..',  sector_ID)
    ''' --------------------------------------
    Calculate min-max
    ------------------------------------------ '''
    region_df_min = region_ds.min(dim=('x','y')).to_dataframe().reset_index()
    region_df_max = region_ds.max(dim=('x','y')).to_dataframe().reset_index()

    ''' min/max over all years, per sector '''
    df_min = pd.DataFrame(region_df_min.min(),index=region_df_min.columns,columns=[sector_ID])
    df_min_list.append(df_min)
    df_max = pd.DataFrame(region_df_max.max(),index=region_df_max.columns,columns=[sector_ID])
    df_max_list.append(df_max)

    ''' median and quantiles '''
    region_q5_list.append( region_ds.chunk(dict(x=-1,y=-1,time=-1)).quantile([0.05]).to_dataframe().transpose().rename(columns={0.05:sector_ID}) )
    region_q95_list.append( region_ds.chunk(dict(x=-1,y=-1,time=-1)).quantile([0.95]).to_dataframe().transpose().rename(columns={0.95:sector_ID}) )
    # region_med_list.append( region_ds.median(dim=('x','y','time')).expand_dims('scalar').to_dataframe().transpose().rename(columns={0:sector_ID}) )

print('--- MIN ---')
display(pd.concat(df_min_list, axis=1))
print('--- MAX ---')
display(pd.concat(df_max_list, axis=1))


# print('--- MEDIAN ---')
# display(pd.concat(region_med_list, axis=1))
print('--- pct 0.05 ---')
display(pd.concat(region_q5_list, axis=1))
print('--- pct 0.95 ---')
display(pd.concat(region_q95_list, axis=1))

.. ASE
.. BSE
.. EIS
.. RS
.. WIS-a
.. WIS-b


/Users/tud500158/opt/anaconda3/envs/NERD2/lib/python3.9/site-packages/dask/array/reductions.py:579: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(x_chunk, axis=axis, keepdims=keepdims)
/Users/tud500158/opt/anaconda3/envs/NERD2/lib/python3.9/site-packages/dask/array/reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


.. WS
--- MIN ---


,ASE,BSE,EIS,RS,WIS-a,WIS-b,WS
time,2015.000000,2015.000000,2015.000000,2015.000000,2015.000000,2015.000000,2015.000000
spatial_ref,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
vx,-4927.189453,-1875.046997,-439.634857,-892.990051,-310.038025,-636.371277,-1271.595947
vy,-4168.872559,-693.820374,-2366.059326,-1552.833374,-269.804321,-769.570068,-703.319336
rema,1.123147,0.003377,0.155419,0.002843,29.149340,27.480970,0.000000
v,0.795817,0.822546,1.957420,0.111137,1.797662,0.248800,0.046153
emax_1px,-0.045136,-0.015093,-0.007066,-0.011579,-0.002244,-0.006078,-0.007557
emin_1px,-0.445389,-0.140252,-0.111792,-0.086256,-0.290649,-0.066868,-0.099082
e_eff_1px,0.000041,0.000123,0.000244,0.000032,0.000045,0.000138,0.000021
elon_1px,-0.432940,-0.072657,-0.061884,-0.081898,-0.064953,-0.047607,-0.050105


--- MAX ---


,ASE,BSE,EIS,RS,WIS-a,WIS-b,WS
time,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000
spatial_ref,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
vx,360.376099,301.415588,1778.739380,344.372131,1939.642456,1400.217896,542.497559
vy,851.362122,1050.854614,433.213409,175.690247,2207.892822,672.346558,1433.908813
rema,243.925751,281.982178,273.577301,726.255310,320.603027,483.688751,0.000000
v,5449.967285,1911.546021,2368.441406,1733.744385,2702.339111,1489.557861,1653.790771
emax_1px,0.310608,0.159163,0.217318,0.079125,0.252160,0.062566,0.092906
emin_1px,0.031803,0.005763,0.033956,0.012834,0.016011,0.012650,0.009731
e_eff_1px,0.443163,0.137843,0.212929,0.076759,0.274454,0.065684,0.088423
elon_1px,0.309752,0.111777,0.210792,0.051863,0.215008,0.054680,0.079958


--- pct 0.05 ---


quantile,ASE,BSE,EIS,RS,WIS-a,WIS-b,WS
vx,-1869.220020,-311.348273,-74.864086,-377.445241,-46.835787,-49.522002,-839.248141
vy,-1744.879297,-264.922266,-1041.953735,-963.579712,6.953736,-511.109288,-0.371292
rema,7.186227,9.062593,4.282541,0.214906,44.255024,46.167946,0.000000
v,38.239758,21.027771,38.149300,33.343252,45.677011,35.059330,26.582656
emax_1px,-0.000922,-0.000962,0.000671,0.000402,0.000230,0.000521,0.000371
emin_1px,-0.060878,-0.020369,-0.037791,-0.007493,-0.010205,-0.020196,-0.011523
e_eff_1px,0.001714,0.001194,0.001425,0.000584,0.000658,0.000978,0.000775
elon_1px,-0.020117,-0.011917,-0.006590,-0.001960,-0.002425,-0.005323,-0.002136
etrans_1px,-0.017875,-0.007743,-0.015315,-0.002690,-0.004940,-0.008344,-0.004240
eshear_1px,-0.025406,-0.009738,-0.015057,-0.003167,-0.005499,-0.010242,-0.007137


--- pct 0.95 ---


quantile,ASE,BSE,EIS,RS,WIS-a,WIS-b,WS
vx,79.959744,176.553323,1353.265961,189.767745,306.489697,955.539102,238.344117
vy,208.776582,238.534466,140.165077,6.868920,346.396138,266.190880,1190.310645
rema,60.679531,85.517494,131.218246,47.468338,95.653008,169.602844,0.000000
v,3467.018445,404.204503,1569.189392,980.429404,455.550253,1086.576111,1277.267548
emax_1px,0.064977,0.015898,0.053311,0.008143,0.015322,0.022882,0.015054
emin_1px,0.000602,0.000172,0.003443,0.000784,0.001241,0.003172,0.000872
e_eff_1px,0.088283,0.021557,0.052691,0.008973,0.015499,0.025126,0.015260
elon_1px,0.029630,0.009649,0.031689,0.004343,0.010633,0.015041,0.008807
etrans_1px,0.013711,0.003948,0.014768,0.002097,0.003540,0.007713,0.002711
eshear_1px,0.017451,0.006714,0.030687,0.004312,0.006387,0.011966,0.005049


### AIS wide calculations

In [14]:

xvar_list=['time','vx','vy','rema', 'v', 'emax_1px', 'emin_1px', 'e_eff_1px', 'elon_1px', 'etrans_1px', 'eshear_1px', 'dEmax_1px', 'deltaV']

data_pxs_df_list=[]
for sector_ID,region_ds in zip(sector_ID_list,region_ds_list): 
    ''' ----------------
    Convert to dataFrame
    --------------------'''

    ''' Aggregate to 1D '''
    region_ds_1d = region_ds.stack(samples=['x','y']) # (time, samples)
    # print('Stacked {} pixels, convert to dataFrame'.format(len(region_ds_1d.samples)))

    data_pxs_df = region_ds_1d.to_dataframe() # nested dataframe
    data_pxs_df = data_pxs_df.rename(columns={"x": "x_coord", "y": "y_coord"}) # rename the column so that flattening the multi-index in the next step does not give an error (relevant for  pandas version > 1.4.3)

    # Flatten the nested multi-index to just column values -- automatically generates a 'year' value for every sample
    data_pxs_df = data_pxs_df.reset_index(level=['time','x','y']) # 18767504 rows; 
    
    # # Drop spatial ref (has not data acutally) 
    data_pxs_df = data_pxs_df.drop(['spatial_ref'],axis=1)

    ''' ----------------
    Drop NaN pixels:
    Pandas drops all rows that contain missing values. 
    - This means that if any variable has a NaN value, that px is dropped.
        So make sure to fill NaN values for variables before this step iif needed (filling gaps etc) 
    - Since I have rows for px per year, this means that if I would have clipped the data to annual ice shelf polygons, the number of pixels per year can vary.  
    -------------------- '''

    # print('.. dropping {} rows with any NaN value'.format( data_pxs_df.isna().sum(axis='index').max() ))
    # data_pxs_df.dropna(subset=xvar_list, axis='index',inplace=True) # Drop rows which contain missing values.


    data_pxs_df_list.append(data_pxs_df[xvar_list])

data_pxs_ais = pd.concat(data_pxs_df_list)

In [15]:
data_pxs_ais.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
time,1138300.0,2016.500000,1.118034,2015.000000,2015.750000,2016.500000,2017.250000,2018.000000
vx,69064.0,-74.141281,354.249451,-4927.189453,-187.877594,-38.871765,56.774933,1939.642456
vy,70965.0,-16.463999,502.252563,-4168.872559,-186.104889,0.000000,141.214645,2207.892822
rema,69872.0,21.276188,36.672546,0.000000,0.000000,0.150604,32.432968,726.255310
v,69064.0,450.493408,433.081543,0.046153,123.154343,325.718384,695.510437,5449.967285
emax_1px,67345.0,0.005549,0.011818,-0.045136,0.001327,0.002461,0.005471,0.310608
emin_1px,67345.0,-0.003947,0.011343,-0.445389,-0.004035,-0.001262,-0.000172,0.033956
e_eff_1px,67345.0,0.006384,0.012975,0.000021,0.001617,0.002906,0.006412,0.443163
elon_1px,67345.0,0.002093,0.010044,-0.432940,0.000202,0.001324,0.002876,0.309752
etrans_1px,67345.0,-0.000490,0.006184,-0.290481,-0.001207,0.000000,0.000719,0.166562


In [16]:
display( pd.DataFrame(data_pxs_ais.quantile([0.05,0.95])).transpose().rename(columns={0.05:'AIS q0.05',0.95:'AIS q0.95'}) )


,AIS q0.05,AIS q0.95
time,2015.000000,2018.000000
vx,-615.050781,354.615021
vy,-845.528076,963.988098
rema,0.000000,90.917084
v,29.490149,1166.208618
emax_1px,0.000235,0.019706
emin_1px,-0.015838,0.001014
e_eff_1px,0.000765,0.021605
elon_1px,-0.004688,0.011001
etrans_1px,-0.005915,0.004312
